In [ ]:
targetPath="drive/MyDrive/twitter/onechunk/"

#Get Dataset.csv file (here is first chunk)



In [ ]:
%%capture
!apt-get install lzop

In [ ]:
!cp "drive/MyDrive/twitter/onechunk/part-00000.lzo" file.lzo

In [ ]:
!lzop -x file.lzo
!mv file Dataset.csv

In [ ]:
import gc
import numpy as np
np.random.seed(1234)
gc.collect()
all_features = ["text_tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains",\
                "tweet_type","language", "tweet_timestamp", "engaged_with_user_id", "engaged_with_user_follower_count",\
               "engaged_with_user_following_count", "engaged_with_user_is_verified", "engaged_with_user_account_creation",\
               "engaging_user_id", "engaging_user_follower_count", "engaging_user_following_count", "engaging_user_is_verified",\
               "engaging_user_account_creation", "engagee_follows_engager","reply_timestamp", "retweet_timestamp", "retweet_with_comment_timestamp", "like_timestamp"]
all_features_to_idx = dict(zip(all_features, range(len(all_features))))

#Load with pandas and train/test splitting

In [ ]:
import pandas as pd
import os
df= pd.read_csv("Dataset.csv",sep='\x01',header=None)
df.columns=all_features

In [ ]:
gc.collect()

185

##Target classes encode

In [ ]:
df["like"]=df["like_timestamp"].apply(lambda row: 1 if row>0 else 0).astype("uint8")
df.drop("like_timestamp",axis=1,inplace=True)

df["retweet"]=df["retweet_timestamp"].apply(lambda row: 1 if row>0 else 0).astype("uint8")
df.drop("retweet_timestamp",axis=1,inplace=True)

df["retweetCom"]=df["retweet_with_comment_timestamp"].apply(lambda row: 1 if row>0 else 0).astype("uint8")
df.drop("retweet_with_comment_timestamp",axis=1,inplace=True)

df["reply"]=df["reply_timestamp"].apply(lambda row: 1 if row>0 else 0).astype("uint8")
df.drop("reply_timestamp",axis=1,inplace=True)
gc.collect()
df["interaction"]=df["reply"]+df["like"]+df["retweet"]+df["retweetCom"]
df["interaction"]=df["interaction"].astype("uint8")

##Divide media types

In [ ]:
def countMedia(arr,mediaType):
    c=0
    for m in arr:
        if m==mediaType:
            c+=1
    return c
df.fillna(value={"present_media":""},inplace=True)
df["photos"]=df["present_media"].apply(lambda row: countMedia((row.split("\t")),"Photo")).astype("uint8")
df["gifs"]=df["present_media"].apply(lambda row: countMedia((row.split("\t")),"GIF")).astype("uint8")
df["videos"]=df["present_media"].apply(lambda row: countMedia((row.split("\t")),"Video")).astype("uint8")
gc.collect()


53

##Replace text tokens , link, media etc with their count

In [ ]:
df["text_tokens"]=df["text_tokens"].apply(lambda row: len(row.split("\t")) if row!="" else 0).astype("uint16")
df.fillna(value={"hashtags":""},inplace=True)
df["hashtags"]=df["hashtags"].apply(lambda row: len(row.split("\t")) if row!="" else 0).astype("uint8")

df["present_media"]=df["present_media"].apply(lambda row: len(row.split("\t")) if row!="" else 0).astype("uint8")
df.fillna(value={"present_links":""},inplace=True)
df["present_links"]=df["present_links"].apply(lambda row: len(row.split("\t")) if row!="" else 0).astype("uint8")
df.fillna(value={"present_domains":""},inplace=True)
df["present_domains"]=df["present_domains"].apply(lambda row: len(row.split("\t")) if row!="" else 0).astype("uint8")
gc.collect()

0

##Drop columns i don't use

In [ ]:
df.drop("tweet_id",axis=1,inplace=True)
df.drop("engaging_user_account_creation",axis=1,inplace=True)
df.drop("engaged_with_user_account_creation",axis=1,inplace=True)
#df.drop("tweet_timestamp",axis=1,inplace=True)
gc.collect()

50

##one hot encode tweet type


In [ ]:
df[:5]

,text_tokens,hashtags,present_media,present_links,present_domains,tweet_type,language,engaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engagee_follows_engager,like,retweet,retweetCom,reply,interaction,photos,gifs,videos
0,50,0,0,0,0,Retweet,488B32D24BD4BB44172EB981C1BCA6FA,BC9980C5385A68894C2862F41884FB27,2473,662,False,2AEB7AFFD36CA43C0C963F24F75BCCAC,169,339,False,False,0,0,0,0,0,0,0,0
1,78,0,0,1,1,TopLevel,B0FA488F2911701DD8EC5B1EA5E322D8,C0EBA033A5C0D2AD91B4500958E2C633,4418640,228,True,865C52D28FA84E5E233B530C6FDAECC2,393,1190,False,False,0,0,0,0,0,0,0,0
2,39,0,0,1,1,TopLevel,B0FA488F2911701DD8EC5B1EA5E322D8,03EA05E7440140ABF047BC216CE34917,219715,3685,True,FE83E568BD4B2ACDBAF876CFBB6C7296,629,1473,False,False,1,0,0,0,1,0,0,0
3,40,0,0,0,0,Retweet,B0FA488F2911701DD8EC5B1EA5E322D8,2700B9E175757F126C082D6A5742787D,2388283,13511,True,DB0201BF93E4CCA1B55932AC36923E43,123,200,False,False,0,0,0,0,0,0,0,0
4,66,0,1,0,0,TopLevel,E7F038DE3EAD397AEC9193686C911677,B7E441F2D886830FB3F0CE29CB35CB6B,414,720,False,04F45C4A769FE2D7C792A96C92342DC2,134,379,False,True,1,0,0,0,1,1,0,0


In [ ]:
df=pd.get_dummies(df,columns=["tweet_type"])

In [ ]:
df[:5]

,text_tokens,hashtags,present_media,present_links,present_domains,language,engaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engagee_follows_engager,like,retweet,retweetCom,reply,interaction,photos,gifs,videos,tweet_type_Quote,tweet_type_Retweet,tweet_type_TopLevel
0,50,0,0,0,0,488B32D24BD4BB44172EB981C1BCA6FA,BC9980C5385A68894C2862F41884FB27,2473,662,False,2AEB7AFFD36CA43C0C963F24F75BCCAC,169,339,False,False,0,0,0,0,0,0,0,0,0,1,0
1,78,0,0,1,1,B0FA488F2911701DD8EC5B1EA5E322D8,C0EBA033A5C0D2AD91B4500958E2C633,4418640,228,True,865C52D28FA84E5E233B530C6FDAECC2,393,1190,False,False,0,0,0,0,0,0,0,0,0,0,1
2,39,0,0,1,1,B0FA488F2911701DD8EC5B1EA5E322D8,03EA05E7440140ABF047BC216CE34917,219715,3685,True,FE83E568BD4B2ACDBAF876CFBB6C7296,629,1473,False,False,1,0,0,0,1,0,0,0,0,0,1
3,40,0,0,0,0,B0FA488F2911701DD8EC5B1EA5E322D8,2700B9E175757F126C082D6A5742787D,2388283,13511,True,DB0201BF93E4CCA1B55932AC36923E43,123,200,False,False,0,0,0,0,0,0,0,0,0,1,0
4,66,0,1,0,0,E7F038DE3EAD397AEC9193686C911677,B7E441F2D886830FB3F0CE29CB35CB6B,414,720,False,04F45C4A769FE2D7C792A96C92342DC2,134,379,False,True,1,0,0,0,1,1,0,0,0,0,1


##Check types

In [ ]:
df.dtypes

text_tokens                          uint16
hashtags                              uint8
present_media                         uint8
present_links                         uint8
present_domains                       uint8
language                             object
engaged_with_user_id                 object
engaged_with_user_follower_count      int64
engaged_with_user_following_count     int64
engaged_with_user_is_verified          bool
engaging_user_id                     object
engaging_user_follower_count          int64
engaging_user_following_count         int64
engaging_user_is_verified              bool
engagee_follows_engager                bool
like                                  uint8
retweet                               uint8
retweetCom                            uint8
reply                                 uint8
interaction                           uint8
photos                                uint8
gifs                                  uint8
videos                          

##Display head

In [ ]:
df.head()

,text_tokens,hashtags,present_media,present_links,present_domains,language,engaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engagee_follows_engager,like,retweet,retweetCom,reply,interaction,photos,gifs,videos,tweet_type_Quote,tweet_type_Retweet,tweet_type_TopLevel
0,50,0,0,0,0,488B32D24BD4BB44172EB981C1BCA6FA,BC9980C5385A68894C2862F41884FB27,2473,662,False,2AEB7AFFD36CA43C0C963F24F75BCCAC,169,339,False,False,0,0,0,0,0,0,0,0,0,1,0
1,78,0,0,1,1,B0FA488F2911701DD8EC5B1EA5E322D8,C0EBA033A5C0D2AD91B4500958E2C633,4418640,228,True,865C52D28FA84E5E233B530C6FDAECC2,393,1190,False,False,0,0,0,0,0,0,0,0,0,0,1
2,39,0,0,1,1,B0FA488F2911701DD8EC5B1EA5E322D8,03EA05E7440140ABF047BC216CE34917,219715,3685,True,FE83E568BD4B2ACDBAF876CFBB6C7296,629,1473,False,False,1,0,0,0,1,0,0,0,0,0,1
3,40,0,0,0,0,B0FA488F2911701DD8EC5B1EA5E322D8,2700B9E175757F126C082D6A5742787D,2388283,13511,True,DB0201BF93E4CCA1B55932AC36923E43,123,200,False,False,0,0,0,0,0,0,0,0,0,1,0
4,66,0,1,0,0,E7F038DE3EAD397AEC9193686C911677,B7E441F2D886830FB3F0CE29CB35CB6B,414,720,False,04F45C4A769FE2D7C792A96C92342DC2,134,379,False,True,1,0,0,0,1,1,0,0,0,0,1


In [ ]:
df.describe()

,text_tokens,hashtags,present_media,present_links,present_domains,engaged_with_user_follower_count,engaged_with_user_following_count,engaging_user_follower_count,engaging_user_following_count,like,retweet,retweetCom,reply,interaction,photos,gifs,videos,tweet_type_Quote,tweet_type_Retweet,tweet_type_TopLevel
count,3.033902e+06,3.033902e+06,3.033902e+06,3.033902e+06,3.033902e+06,3.033902e+06,3.033902e+06,3.033902e+06,3.033902e+06,3.033902e+06,3.033902e+06,3.033902e+06,3.033902e+06,3.033902e+06,3.033902e+06,3.033902e+06,3.033902e+06,3.033902e+06,3.033902e+06,3.033902e+06
mean,4.577666e+01,3.958948e-01,5.541504e-01,1.160591e-01,1.160591e-01,7.528097e+05,3.622267e+03,8.012008e+02,7.056084e+02,3.980313e-01,8.751766e-02,7.051975e-03,2.962027e-02,5.222212e-01,4.716405e-01,1.259269e-02,6.991722e-02,8.148022e-02,3.282245e-01,5.902953e-01
std,2.930063e+01,1.115992e+00,8.919881e-01,3.428927e-01,3.428927e-01,4.450692e+06,3.318459e+04,9.863545e+03,1.672492e+03,4.894920e-01,2.825922e-01,8.367943e-02,1.695374e-01,5.457068e-01,8.931532e-01,1.115202e-01,2.550682e-01,2.735712e-01,4.695671e-01,4.917793e-01
min,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.500000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.180000e+02,2.030000e+02,7.000000e+01,1.530000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,3.900000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.607000e+03,5.660000e+02,2.100000e+02,3.340000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
75%,5.900000e+01,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,8.824000e+04,1.475000e+03,5.760000e+02,7.330000e+02,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
max,4.240000e+02,4.300000e+01,4.000000e+00,1.000000e+01,1.000000e+01,1.292405e+08,4.256371e+06,5.358904e+06,9.329220e+05,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,4.000000e+00,4.000000e+00,3.000000e+00,4.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


##Split dataset

In [ ]:
 msk = np.random.rand(len(df)) < 0.8
 
 train=df[msk]

 test=df[~msk]

In [ ]:
train.to_csv("train.csv", header=train.columns)
test.to_csv("test.csv", header=test.columns)

In [ ]:
!bzip2 -c --best train.csv > train.bz2
!bzip2 -c --best test.csv > test.bz2

In [ ]:
os.system(f'cp train.bz2 "{targetPath}/train.bz2"')
os.system(f'cp test.bz2 "{targetPath}/test.bz2"')

0

#Prepare for Cugraph

In [ ]:
trainCu=pd.DataFrame(train.loc[df["interaction"]>0])


In [ ]:
for col in trainCu.columns:
    if col not in ["engaged_with_user_id","engaging_user_id","interaction"]:
        trainCu.drop(col,axis=1,inplace=True)

In [ ]:
trainCu["interactions"]=trainCu.groupby(["engaged_with_user_id","engaging_user_id"])['interaction'].transform('sum')

In [ ]:
trainCu= trainCu.drop_duplicates(subset= ["engaged_with_user_id","engaging_user_id"]).drop("interaction",axis=1)

In [ ]:
trainCu[:5]

,engaged_with_user_id,engaging_user_id,interactions
2,03EA05E7440140ABF047BC216CE34917,FE83E568BD4B2ACDBAF876CFBB6C7296,1
4,B7E441F2D886830FB3F0CE29CB35CB6B,04F45C4A769FE2D7C792A96C92342DC2,1
7,223797E039AFAC669C3A24A43B59CD5E,0F7B9515737C440274E43E09E01FF8CF,1
11,503B6E875783CABBC0F4CB7FEEA54622,C63E79394F6A9CCB4D2F85A200083F6E,1
12,D77A8DF6804617BFD165C4A11304A27A,7EE0A31E5381C7CF4A572F29F6FBE625,1


In [ ]:
trainCu.columns

Index(['engaged_with_user_id', 'engaging_user_id', 'interactions'], dtype='object')

In [ ]:
trainCu.to_csv("trainCu.csv", header=trainCu.columns)

In [ ]:
!bzip2 -c --best trainCu.csv > trainCu.bz2

In [ ]:
os.system(f'cp trainCu.bz2 "{targetPath}/trainCu.bz2"')

0